In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
def ex_tag(sid, page):
    ### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###
    
    ## 1.
    url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}"\
    "#&date=%2000:00:00&page={page}"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    a_tag = soup.find_all("a")
    
    ## 2.
    tag_lst = []
    for a in a_tag:
        if "href" in a.attrs:  # href가 있는것만 고르는 것
            if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                tag_lst.append(a["href"])
                
    return tag_lst

In [3]:
def re_tag(sid):
    ### 특정 분야의 100페이지까지의 뉴스의 링크를 수집하여 중복 제거한 리스트로 변환하는 함수 ###
    ## 여기 바꾸면 돼  예정!!!!
    re_lst = []
    for i in tqdm(range(1,1000)):
        lst = ex_tag(sid, i+1)
        re_lst.extend(lst)

    # 중복 제거
    re_set = set(re_lst)
    re_lst = list(re_set)
    
    return re_lst

In [4]:
all_hrefs = {}
sids = [i for i in range(100,106)]  # 분야 리스트
### 분야 바꾸고 싶으면 여기 바꾸면 됨
# 각 분야별로 링크 수집해서 딕셔너리에 저장
for sid in sids:
    sid_data = re_tag(sid)
    all_hrefs[sid] = sid_data

  0%|          | 0/999 [00:00<?, ?it/s]

  0%|          | 0/999 [00:00<?, ?it/s]

  0%|          | 0/999 [00:00<?, ?it/s]

  0%|          | 0/999 [00:00<?, ?it/s]

  0%|          | 0/999 [00:00<?, ?it/s]

  0%|          | 0/999 [00:00<?, ?it/s]

In [5]:
def art_crawl(all_hrefs, sid, index):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Args: 
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학]
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    art_dic = {}
    
    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    url = all_hrefs[sid][index]
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    
    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str
    
    return art_dic

In [6]:
# 모든 섹션의 데이터 수집 (제목, 날짜, 본문, section, url)
section_lst = [s for s in range(100, 106)]
artdic_lst = []

for section in tqdm(section_lst):
    for i in tqdm(range(len(all_hrefs[section]))):
        art_dic = art_crawl(all_hrefs, section, i)
        art_dic["section"] = section
        art_dic["url"] = all_hrefs[section][i]
        artdic_lst.append(art_dic)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/725 [00:00<?, ?it/s]

  0%|          | 0/986 [00:00<?, ?it/s]

  0%|          | 0/979 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/188 [00:00<?, ?it/s]

In [7]:
import pandas as pd

art_df = pd.DataFrame(artdic_lst)

In [8]:
art_df.to_csv("article_df.csv")

In [9]:
art_df.sort_values(by='date', ascending=True)

,title,date,main,section,url
368,[단독] 검사 출신의 140쪽 검토보고서에 발칵 뒤집힌 감사원,2023.06.27. 오전 11:52,"""언어의 가능한 의미는 법률해석의 한계이며 문언 해석이 법률해석의 기본임"" 얼마 ...",100,https://n.news.naver.com/mnews/article/056/001...
697,"""전형적 포퓰리즘"" TV 수신료 분리징수에 학계도 집단반발",2023.06.27. 오전 11:53,"3대 언론학회 다음주 긴급 토론회... ""수신료 납부원과 싸우고 가산금까지, 국민불...",100,https://n.news.naver.com/mnews/article/047/000...
633,"정부 ""日오염수 검토 마무리단계…기준 초과 검출 핵종은 5개""",2023.06.27. 오전 11:54,"원안위원장 ""어제 한일 기술회의…방사선 감시기 추가설치 확인""수산물 안전관리 예비비...",100,https://n.news.naver.com/mnews/article/001/001...
713,"윤, 새만금 6.6兆 투자 유치 성과 강조… 총선 앞두고 경제 올인",2023.06.27. 오전 11:55,윤재옥(가운데) 국민의힘 원내대표가 27일 오전 국회에서 열린 원내대책회의에 참석해...,100,https://n.news.naver.com/mnews/article/021/000...
605,내일부터 ‘만 나이’ 시행… 술·담배 살 땐 ‘연 나이’,2023.06.27. 오전 11:55,행정·민사상 1∼2세 어려져윤석열 정부의 주요 국정과제로 추진된 ‘만 나이 통일법’...,100,https://n.news.naver.com/mnews/article/021/000...
...,...,...,...,...,...
3296,"""강남 터줏대감은 나""...삼성 '애플 강남' 인근에 '삼성 강남' 문연다",2023.06.28. 오후 2:41,"강남역 인근 체험형 플래그십 스토어오는 29일 오픈, 총 6개층 605평 규모\n\...",105,https://n.news.naver.com/mnews/article/008/000...
3254,"개보위, ‘개인정보 유출’ 엘지헬로비전에 11억여 원 과징금 부과",2023.06.28. 오후 2:44,"개인정보위가 이용자 4만여 명의 개인정보를 유출시킨 엘지헬로비전에 대해 11억 3,...",105,https://n.news.naver.com/mnews/article/056/001...
3268,"정부, 주유소 내 연료전지 구축 전력생산 허용",2023.06.28. 오후 2:44,실증사업 종료 후 2000여 개 에너지슈퍼스테이션 구축 투자계획 구체화정부가 기존에...,105,https://n.news.naver.com/mnews/article/092/000...
3207,"삼성전자·LG헬로비전, 개인정보 유출…과징금 9억·11억 부과",2023.06.28. 오후 2:45,고학수 개인정보보호위원회 위원장이 28일 오전 서울 종로구 정부서울청사에서 열린 제...,105,https://n.news.naver.com/mnews/article/016/000...


In [10]:
art_df.sort_values

<bound method DataFrame.sort_values of                                           title                  date  \
0            野, '日 오염수 방류 반대' 결의안 단독 처리…與, 집단퇴장   2023.06.27. 오후 2:59   
1         '86운동권' 함운경, 야당 오염수 공세에 "반일 부추기려는 의도"   2023.06.28. 오전 9:38   
2                 이낙연, 첫 행보로 DJ묘역 참배…“제 정치의 원점”  2023.06.28. 오전 11:15   
3         국힘, 오늘 '후쿠시마 괴담' 대응…함운경 강연·우리바다TF 간담회   2023.06.28. 오전 6:05   
4             국방부 "병역의무 이행 연령 기준 '만 나이' 적용 안 해"  2023.06.28. 오전 11:02   
...                                         ...                   ...   
3294       SKT, 하나금융그룹과 ‘찾아가는 AI 교육 프로그램’ 공동 운영  2023.06.28. 오전 11:30   
3295        MZ 겨냥 '삼성 강남' 오픈..."화장실 쓰러 오셔도 됩니다"   2023.06.28. 오후 2:28   
3296  "강남 터줏대감은 나"...삼성 '애플 강남' 인근에 '삼성 강남' 문연다   2023.06.28. 오후 2:41   
3297             애플 견제하는 '삼성 강남'…애플 강남 인접지역 문연다   2023.06.28. 오후 2:10   
3298  개인정보위, '삼쩜삼'에 과징금 8억 부과…"법적 근거없이 주민번호 수집"   2023.06.28. 오후 2:47   

                                                   main  section  \
0     與 "농해수위 표결